In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import initializers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adagrad, RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

verbose = 1
epochs = 9999


path = '~/Documents/abhishek/assignment/Group_6/'

# import logging

# logging.getLogger('tensorflow').setLevel(logging.WARNING)


dict = {0: [1, 0, 0, 0, 0], 3: [0, 1, 0, 0, 0], 4: [0, 0, 1, 0, 0], 8: [0, 0, 0, 1, 0], 9: [0, 0, 0, 0, 1]}

set = ['train', 'val', 'test']

X = []

for t in range(0, 3):
     x = []
     for i in dict:
         new_path = path + f'{set[t]}/{i}/'
         for k in os.listdir(new_path):
             x.append(cv2.imread(new_path + k, cv2.IMREAD_GRAYSCALE).flatten())
     X.append(x)

X_train = np.array(X[0])
X_val = np.array(X[1])
X_test = np.array(X[2])

l1 = X_train.shape[0]//5
l2 = X_val.shape[0]//5
l3 = X_test.shape[0]//5

def ground_truth(l):
    
    gt = []
    for i in dict:
        gt += l*[dict[i]]

    return gt

y_train = np.array(ground_truth(l1))
y_val = np.array(ground_truth(l2))
y_test = np.array(ground_truth(l3))

print(X_train.shape, X_val.shape, X_test.shape)

print(tf.__version__)

def architecture(hidden):

    tf.random.set_seed(42)
    np.random.seed(42)

    if len(hidden) == 3:
      model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,), name='Input_Layer'),
      tf.keras.layers.Dense(hidden[0], activation='sigmoid', name='HL1'),
      tf.keras.layers.Dense(hidden[1], activation='sigmoid', name='HL2'),
      tf.keras.layers.Dense(hidden[2], activation='sigmoid', name='HL3'),
      tf.keras.layers.Dense(5, activation='sigmoid', name='Output_Layer')
      ])

    if len(hidden) == 4:
      model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,), name='Input_Layer'),
      tf.keras.layers.Dense(hidden[0], activation='sigmoid', name='HL1'),
      tf.keras.layers.Dense(hidden[1], activation='sigmoid', name='HL2'),
      tf.keras.layers.Dense(hidden[2], activation='sigmoid', name='HL3'),
      tf.keras.layers.Dense(hidden[3], activation='sigmoid', name='HL4'),
      tf.keras.layers.Dense(5, activation='sigmoid', name='Output_Layer')
      ])

    if len(hidden) == 5:
      model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,), name='Input_Layer'),
      tf.keras.layers.Dense(hidden[0], activation='sigmoid', name='HL1'),
      tf.keras.layers.Dense(hidden[1], activation='sigmoid', name='HL2'),
      tf.keras.layers.Dense(hidden[2], activation='sigmoid', name='HL3'),
      tf.keras.layers.Dense(hidden[3], activation='sigmoid', name='HL4'),
      tf.keras.layers.Dense(hidden[4], activation='sigmoid', name='HL5'),
      tf.keras.layers.Dense(5, activation='sigmoid', name='Output_Layer')
      ])

    return model

def save_w(hidden):
    model = architecture(hidden)
    model.save_weights('./weights/'+f'model_{str(hidden)}.h5')


def conv_label(y_true, y_pred):

  y_pred = to_categorical(np.argmax(y_pred, axis=1), 5)

  label = tf.argmax(y_true, axis = 1)
  pred = tf.argmax(y_pred, axis = 1)

  return label, pred

def cm_plot(hidden, conf, string):
    
    print('\n',string)
    print(conf)
    #import seaborn as sb

    #x_labels = ['0', '3', '4', '8', '9']
    #y_labels = ['0', '3', '4', '8', '9']

    #sb.heatmap(conf, annot = True, fmt='g',xticklabels = x_labels, yticklabels = y_labels, cbar=False)
    
    #fig = plt.figure(figsize=(12,12), num = 3)
    
    #plt.title('Actual Digit', fontsize = 10)
    #plt.xlabel(f'{string} {hidden}', fontsize = 12)
    #plt.ylabel('Predicted Digit', fontsize = 10)
    #plt.tick_params(axis='both', which='major', labelsize=8, labelbottom = False, bottom=False, top = False, labeltop=True)
    
    #plt.savefig('./output/'+string+f'_{hidden}.png')
    #plt.show()
    #plt.clf()
    #plt.close(fig)

opt_dict = {0: 'SGD', 1: 'Batch Gradient Descent', 2: 'Momentum', 3: 'NAG', 4: 'Adagrad', 5: 'RMSProp', 6: 'Adam'}

def report(model, hidden):
  print('\nValidation and Test Accuracy:')

  print(model.evaluate(x = X_val, y = y_val, batch_size = None, verbose=0, callbacks = None)[1])
  print(model.evaluate(x = X_test, y = y_test, batch_size = None, verbose=0, callbacks = None)[1], '\n')

  y_pred_train = model.predict(X_train, batch_size=None, verbose=0, callbacks=None)
  y_pred_test = model.predict(X_test, batch_size=None, verbose=0, callbacks=None)

  y_train_cl, y_pred_train_sgd_cl = conv_label(y_train, y_pred_train)
  y_test_cl, y_pred_test_sgd_cl = conv_label(y_test, y_pred_test)

  cm1 = tf.math.confusion_matrix(y_train_cl, y_pred_train_sgd_cl)
  cm_plot(hidden, cm1, 'Training Confusion Matrix')
  
  cm2 = tf.math.confusion_matrix(y_test_cl, y_pred_test_sgd_cl)
  cm_plot(hidden, cm2, 'Test Confusion Matrix')

opt_dict = {0: 'SGD', 1: 'Batch Gradient Descent', 2: 'Momentum', 3: 'NAG', 4: 'Adagrad', 5: 'RMSProp', 6: 'Adam'}
optimizers = [tf.keras.optimizers.SGD(learning_rate = 0.001, name='SGD'), 
              tf.keras.optimizers.SGD(learning_rate=0.001, name='BGD'),
              tf.keras.optimizers.SGD(learning_rate = 0.001, momentum = 0.9, name = 'Momentum_SGD'),
              tf.keras.optimizers.SGD(learning_rate = 0.001, momentum = 0.9, nesterov = True, name='NAG'),
              tf.keras.optimizers.Adagrad(learning_rate = 0.001, epsilon = 1e-08, name = "Adagrad"),
              tf.keras.optimizers.RMSprop(learning_rate = 0.001, rho = 0.99, momentum = 0.0, epsilon = 1e-08, name="RMSProp"),
              tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, name='Adam') ]

my_callbacks = [EarlyStopping(monitor = 'loss', min_delta = 1e-04, patience = 1)]

def train2(hidden):

  save_w(hidden)

  print('\n',hidden, '\n')
  count = 0
  hist_arr = []
  acc_arr = []
  epp = []
  for optimizer in optimizers:

      bs = 1
      if count == 1:
          bs = X_train.shape[0]
      
      model = architecture(hidden)
      model.load_weights('./weights/'+f'model_{hidden}.h5')

      # for layer in model.layers:
      #     print(layer.get_weights()[0])
      #     break

      model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
      model_fit = model.fit(X_train, y_train, batch_size=bs, epochs=epochs, callbacks = my_callbacks, verbose=verbose)
      loss, acc = model.evaluate(X_test, y_test, verbose = 0)
      print("Optimizer: {}, Test Accuracy: {:.2f}%".format(optimizer.__class__.__name__, acc * 100))

      avg_err = model_fit.history['loss']

      hist_arr.append(avg_err)
      acc_arr.append(model_fit.history['accuracy'][len(avg_err) - 1])
      epp.append(len(avg_err))
      print(f'\n-- REPORT: {opt_dict[count]}--')
      report(model, hidden)

      count = count + 1

  print('\nOrder:')
  for i in opt_dict:
      print(f'{opt_dict[i]}')

  print('\nTraining Accuracy: ')
  for i in opt_dict:
      print(f'{acc_arr[i]}')

  print('\nEpochs: ')
  for i in opt_dict:
      print(f'{epp[i]}')
  

  return hist_arr

def plot_epoch(hidden, hist_arr):

    #plt.figure(num = 1)
   
    #plt.plot(hist_arr[1])
    #plt.title("Average Error vs Epoch")
    #plt.xlabel("Epoch")
    #plt.ylabel("Average Error")
    #plt.legend(['Batch Gradient Descent'], fontsize = 7, loc = 'upper right')
    #plt.savefig('./output/'+f'bgd_{hidden}.png')
    #plt.clf()
    #plt.close()

    #leg = ['SGD', 'Batch-GD', 'Momentum', 'NAG', 'Adagrad', 'RMSProp', 'Adam']

    leg = ['SGD', 'Momentum', 'NAG', 'Adagrad', 'RMSProp', 'Adam']
    
    #plt.figure(num = 2)
    #x = hist_arr.pop(1)
    for i in range(len(hist_arr)):
        plt.plot(hist_arr[i])
   
   	
    plt.title("Average Error vs Epoch "+str(hidden))
    plt.xlabel("Epoch")
    plt.ylabel("Average Error")
    plt.legend(leg, fontsize = 7, loc = 'upper right')
    plt.savefig('./output/'+f'epoch_{hidden}.png')
    plt.clf()
    plt.close()
    

def run(hidden):
    hist_arr = train2(hidden)
    plot_epoch(hidden, hist_arr)
    
run_list = [(4, 2, 1), (8, 4, 2), (32, 16, 8), (64, 32, 16), (512, 256, 64)]

#run_list = [(8, 4, 2), (32, 16, 8), (64, 32, 16), (512, 256, 64), (1024, 512, 256), (2048, 1024, 512), (1000, 750, 250),  (100, 50, 25),
#             (16, 8, 4, 2), (32, 16, 8, 4), (256, 64, 16, 4), (1024, 512, 256, 64), (2048, 1024, 512, 256), 
#             (64, 32, 16, 8, 4), (512, 64, 32, 16, 8), (2048, 1024, 512, 64, 8), (4096, 2048, 1024, 512, 256)]


for i in run_list:
    run(i)
